Importar librerias  

In [36]:
import io
import re
import string
import tqdm

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

Tokenizacion

In [37]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [38]:
#Crear vocabulario
vocab, index = {}, 1
vocab['pad']= 0  #padding
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index +=1

vocab_size=len(vocab)
print(vocab)


{'pad': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [39]:
# inversa del vocabulario
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: 'pad', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [40]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


Generar skip-grams

In [41]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size= window_size,
    negative_samples=0)
print(len(positive_skip_grams))

for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]},{inverse_vocab[context]}))")

26
(3, 5): (road,in))
(5, 4): (in,shimmered))
(6, 7): (hot,sun))
(3, 1): (road,the))
(4, 2): (shimmered,wide))


Generar muestras negativas

In [42]:
target_word, context_word = positive_skip_grams[0]
num_ns = 4 #Numero de muestras negativas

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1,1))

negative_sampling_candidates, _ , _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,
    num_true=1,
    num_sampled = num_ns,
    unique = True,
    range_max = vocab_size,
    seed = SEED,
    name = "negative_sampling"
)

print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([3 6 0 1], shape=(4,), dtype=int64)
['road', 'hot', 'pad', 'the']


In [43]:
squeezed_context_class = tf.squeeze(context_class, 1)
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)
label = tf.constant([1] + [0] * num_ns, dtype="int64")
target = target_word

print(f"target_index     :{target}")
print(f"target_word      :{inverse_vocab[target_word]}")
print(f"context_indices  :{context}")
print(f"context_words    :{[inverse_vocab[c.numpy()] for c in context]}")
print(f"label            :{label}")

target_index     :3
target_word      :road
context_indices  :[5 3 6 0 1]
context_words    :['in', 'road', 'hot', 'pad', 'the']
label            :[1 0 0 0 0]


Generar datos de entrenamiento

In [44]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  targets, contexts, labels =[],[],[]

  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  for sequence in tqdm.tqdm(sequences):

    #Generar skip_gram positivo de cada palabra
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      sequence,
      vocabulary_size= vocab_size,
      sampling_table=sampling_table,
      window_size=window_size,
      negative_samples=0)

    #Generar muestras negativas
    for target_word, context_word in positive_skip_grams:

      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)

      negative_sampling_candidates, _, _ =tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed = seed,
          name = "negative_sampling")

      # Concatenar y etiquetas
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      #Agregar a la lista
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

Descargar documento (Text Corpus)

In [45]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [46]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x),bool))

Estandarizacion

In [47]:
def custom_stadardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

vocab_size = 4096
sequence_length = 10

vectorize_layer = layers.TextVectorization(
    standardize = custom_stadardization,
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length)

vectorize_layer.adapt(text_ds.batch(1024))

In [48]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [49]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [50]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))
for seq in sequences[:10]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

32777
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[   7   41   34 1286  344    4  200   64    4 3690] => ['you', 'are', 'all', 'resolved', 'rather', 'to', 'die', 'than', 'to', 'famish']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[1286 1286    0    0    0    0    0    0    0    0] => ['resolved', 'resolved', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[  89    7   93 1187  225   12 2442  592    4    2] => ['first', 'you', 'kno

Generar datos de entrenamiento

In [51]:
targets, contexts, labels = generate_training_data(
    sequences = sequences,
    window_size= 2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED
)

targets = np.array(targets)
contexts = np.array(contexts)
labels= np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [01:01<00:00, 534.06it/s]




targets.shape: (64864,)
contexts.shape: (64864, 5)
labels.shape: (64864, 5)
